In [34]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

def coin_flip_inference(num_flips, prior_type, prior_params_alpha, prior_params_beta, likelihood_type):
    # Step 1: Define Prior Beliefs (User-Defined Prior Distribution)
    if prior_type == "Beta":
        prior_params = (prior_params_alpha, prior_params_beta)
        prior_distribution = stats.beta(*prior_params)
        prior_description = f'Prior Alpha: {prior_params_alpha}, Prior Beta: {prior_params_beta}'
        user_text = "Enter Alpha and Beta for the prior:"
    elif prior_type == "Gaussian":
        prior_params = (prior_params_alpha, prior_params_beta)
        prior_distribution = stats.norm(*prior_params)
        prior_description = f'Prior Mean: {prior_params_alpha}, Prior Std Dev: {prior_params_beta}'
        user_text = "Enter Mean and Std Dev for the prior:"
    # Add more elif blocks for other distribution types

    # Step 2: Collect Data
    # Simulate coin flips (0 for tails, 1 for heads)
    data = np.random.choice([0, 1], num_flips)

    # Step 3: Likelihood Function
    def binomial_likelihood(theta):
        num_heads = np.sum(data)
        return stats.binom.pmf(num_heads, num_flips, theta)

    def normal_likelihood(theta):
        return stats.norm.pdf(data, loc=theta, scale=0.5).prod()

    # Select the likelihood function based on user choice
    if likelihood_type == "Binomial":
        likelihood_function = binomial_likelihood
    elif likelihood_type == "Normal":
        likelihood_function = normal_likelihood
    # Add more elif blocks for other likelihood types

    # Step 4: Bayes' Theorem
    # Calculate the posterior distribution
    theta_values = np.linspace(0, 1, 1000)  # Possible values for the parameter θ
    posterior_values = [prior_distribution.pdf(theta) * likelihood_function(theta) for theta in theta_values]

    # Normalize the posterior (divide by the integral)
    posterior_values /= np.trapz(posterior_values, theta_values)

    # Step 5: Analyze the Posterior
    # Calculate summary statistics
    mean_posterior = np.trapz(theta_values * posterior_values, theta_values)
    median_posterior = np.interp(0.5, np.cumsum(posterior_values), theta_values)
    cred_interval = stats.beta.interval(0.95, *prior_params) if prior_type == "Beta" else stats.norm.interval(0.95, *prior_params)
    # Add more elif blocks for other distribution types

    # Step 6: Make Inferences
    print(f"Estimated Probability of Heads (Posterior Mean): {mean_posterior:.3f}")
    print(f"Median of the Posterior: {median_posterior:.3f}")
    print(f"95% Credible Interval: ({cred_interval[0]:.3f}, {cred_interval[1]:.3f})")

    # Plot the prior and posterior distributions
    plt.figure(figsize=(10, 5))
    plt.plot(theta_values, prior_distribution.pdf(theta_values), label='Prior', color='blue')
    plt.plot(theta_values, posterior_values, label='Posterior', color='green')
    plt.xlabel('Parameter (θ)')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.title(f'Bayesian Inference (Number of Flips: {num_flips}, Prior Type: {prior_type}, Likelihood Type: {likelihood_type})\n{prior_description}')
    plt.show()


In [35]:
# Create interactive widgets for changing the number of coin tosses, prior type, prior parameters, and likelihood type
num_flips_widget = widgets.IntSlider(value=1, min=1, max=1001, step=10, description='Number of Flips')
prior_type_widget = widgets.Dropdown(options=["Beta", "Gaussian"], value="Beta", description='Prior Type')
prior_params_alpha_widget = widgets.FloatText(value=2.0, description=f'Prior Alpha')
prior_params_beta_widget = widgets.FloatText(value=2.0, description=f'Prior Beta')
likelihood_type_widget = widgets.Dropdown(options=["Binomial", "Normal"], value="Binomial", description='Likelihood Type')

# Display the interactive plot with user-defined prior and likelihood
interactive_plot = interact(
    coin_flip_inference,
    num_flips=num_flips_widget,
    prior_type=prior_type_widget,
    prior_params_alpha=prior_params_alpha_widget,
    prior_params_beta=prior_params_beta_widget,
    likelihood_type=likelihood_type_widget
)
display(interactive_plot)

interactive(children=(IntSlider(value=1, description='Number of Flips', max=1001, min=1, step=10), Dropdown(de…

<function __main__.coin_flip_inference(num_flips, prior_type, prior_params_alpha, prior_params_beta, likelihood_type)>